## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/89-googleq-a-train-glove-radam/model.h5'
tokenizer_path = '/kaggle/input/89-googleq-a-train-glove-radam/tokenizer.joblib'
category_encoder_path = '/kaggle/input/89-googleq-a-train-glove-radam/category_encoder.joblib'
host_encoder_path = '/kaggle/input/89-googleq-a-train-glove-radam/host_encoder.joblib'
scaler_path = '/kaggle/input/89-googleq-a-train-glove-radam/scaler.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())

    
# Text length
test['question_title_len'] = test['question_title'].apply(lambda x : len(x))
test['question_body_len'] = test['question_body'].apply(lambda x : len(x))
test['answer_len'] = test['answer'].apply(lambda x : len(x))

# Word count
test['question_title_word_cnt'] = test['question_title'].apply(lambda x : len(x.split(' ')))
test['question_body_word_cnt'] = test['question_body'].apply(lambda x : len(x.split(' ')))
test['answer_word_cnt'] = test['answer'].apply(lambda x : len(x.split(' ')))

# Unique word count
test['question_title_unique_word_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' '))))
test['question_body_unique_word_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' '))))
test['answer_unique_word_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' '))))

# Misspelings count
misspelling_mapping_set = set(misspelling_mapping.keys())
test['question_title_misspelings_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['question_body_misspelings_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['answer_misspelings_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))

# Meta features
test['question_title_count'] = test['question_title'].apply(lambda x : x.count('?'))
test['question_body_count'] = test['question_body'].apply(lambda x : x.count('?'))
test['title_exp_count'] = test['question_title'].apply(lambda x : x.count('why'))
test['body_exp_count'] = test['question_body'].apply(lambda x : x.count('why'))
test['title_choice_count'] = test['question_title'].apply(lambda x : x.count('or'))
test['body_choice_count'] = test['question_body'].apply(lambda x : x.count('or'))
test['question_aswer_ratio'] = test['question_title_word_cnt'] / test['answer_word_cnt']
test['same_author'] = test['question_user_page'] == test['answer_user_page']

for feature in text_features:
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 100000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 100
ANSWER_MAX_LEN = 100

## Test set

In [6]:
tokenizer = load(tokenizer_path)
ohe_category = load(category_encoder_path)
ohe_host = load(host_encoder_path)
scaler = load(scaler_path)

# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']
X_test_category = ohe_category.transform(test[['category']]).toarray()
X_test_host = ohe_host.transform(test[['host']]).toarray()
X_test_dense = test[['question_title_len', 'question_title_word_cnt', 'question_title_unique_word_cnt', 'question_title_misspelings_cnt', 
                     'question_body_len', 'question_body_word_cnt', 'question_body_unique_word_cnt', 'question_body_misspelings_cnt', 
                     'answer_len', 'answer_word_cnt', 'answer_unique_word_cnt', 'answer_misspelings_cnt', 
                     'question_title_count', 'question_body_count', 'title_exp_count', 'body_exp_count', 'title_choice_count', 
                     'body_choice_count', 'body_choice_count', 'question_aswer_ratio', 'same_author']]

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test_dense = scaler.transform(X_test_dense)

X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq, X_test_category, X_test_host, X_test_dense]

# Model

In [7]:
# use_embed = hub.load(module_url)

# def USEEmbedding(x):
#     return use_embed(tf.squeeze(tf.cast(x, tf.string)))

# def encoder_block(input_layer):
#     encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
#     return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
embedding_name = 'GloVe'
print('n unknown words "%s": %d' % (embedding_name, len(glove_unknown_words)))
print('"%s" embedding shape: %s' % (embedding_name, glove_embedding_matrix.shape))

n unknown words "GloVe": 19913
"GloVe" embedding shape: (58007, 300)


In [9]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = SpatialDropout1D(0.3)(title)
title = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(title)
title = Dropout(0.5)(title)
title = Dense(512, kernel_initializer='glorot_uniform')(title)
title = BatchNormalization()(title)
title_out = Activation('relu')(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body = SpatialDropout1D(0.3)(body)
body = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='glorot_uniform'))(body)
body = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(body)
body = Dropout(0.5)(body)
body = Dense(512, kernel_initializer='glorot_uniform')(body)
body = BatchNormalization()(body)
body_out = Activation('relu')(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer = SpatialDropout1D(0.3)(answer)
answer = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='glorot_uniform'))(answer)
answer = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(answer)
answer = Dropout(0.5)(answer)
answer = Dense(512, kernel_initializer='glorot_uniform')(answer)
answer = BatchNormalization()(answer)
answer_out = Activation('relu')(answer)

# Category
input_category = Input(shape=(X_test_category.shape[1],), dtype=tf.float32, name='input_category')
category = Embedding(X_test_category.shape[1], X_test_category.shape[1]//2)(input_category)
category_out = GlobalAveragePooling1D()(category)

# Host
input_host = Input(shape=(X_test_host.shape[1],), dtype=tf.float32, name='input_host')
host = Embedding(X_test_host.shape[1], X_test_host.shape[1]//2)(input_host)
host_out = GlobalAveragePooling1D()(host)

# Dense features
input_dense = Input(shape=(X_test_dense.shape[1],), dtype=tf.float32, name='input_dense')

# Output
output = Concatenate()([title_out, body_out, answer_out, category_out, host_out, input_dense])
output = Dropout(0.5)(output)
output = Dense(512, kernel_initializer='glorot_uniform')(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)
output = Dropout(0.5)(output)
output = Dense(512, kernel_initializer='glorot_uniform')(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='glorot_uniform')(output)

model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq, input_category, input_host, input_dense], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_body_seq (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     17402100    input_body_seq[0][0]             
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.872445,0.556143,0.153106,0.622828,0.690130,0.693834,0.613720,0.570255,0.435830,...,0.861079,0.909189,0.639352,0.950673,0.957576,0.800367,0.090577,0.071974,0.787209,0.908146
1,46,0.857223,0.565483,0.002072,0.732841,0.771507,0.936067,0.542943,0.427908,0.060375,...,0.683366,0.922212,0.619986,0.955795,0.965875,0.839641,0.909770,0.142055,0.109683,0.872027
2,70,0.927805,0.719744,0.026110,0.856080,0.939923,0.914456,0.617671,0.629368,0.262324,...,0.885039,0.897176,0.635453,0.947795,0.966656,0.824360,0.026348,0.040018,0.777811,0.872275
3,132,0.820961,0.437168,0.003144,0.718743,0.692086,0.904361,0.530400,0.411751,0.092344,...,0.626518,0.942841,0.635909,0.965442,0.979413,0.840788,0.867663,0.165113,0.519442,0.879593
4,200,0.908007,0.529916,0.007260,0.715337,0.824995,0.855344,0.586053,0.475379,0.259679,...,0.753255,0.927464,0.629063,0.972789,0.964871,0.816688,0.558126,0.141281,0.448786,0.925645


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.873081,0.564346,0.030371,0.719372,0.798404,0.860802,0.570818,0.481569,0.224662,...,0.757583,0.923783,0.646698,0.960567,0.966512,0.846856,0.551185,0.149076,0.503617,0.897946
std,2812.670060,0.042705,0.126202,0.061420,0.101349,0.090719,0.122093,0.039495,0.072358,0.135904,...,0.096675,0.023036,0.030194,0.012418,0.015055,0.033984,0.325132,0.066917,0.270496,0.023799
min,39.000000,0.713014,0.353996,0.000408,0.239658,0.380683,0.243156,0.490246,0.362012,0.021738,...,0.551815,0.816662,0.530375,0.898702,0.873970,0.683528,0.006478,0.009511,0.050494,0.799546
25%,2572.000000,0.841250,0.447154,0.003954,0.675307,0.751410,0.861102,0.540871,0.431349,0.116526,...,0.675771,0.909460,0.628688,0.953661,0.960443,0.828162,0.199377,0.105651,0.271763,0.880808
50%,5093.000000,0.870350,0.545869,0.006770,0.726512,0.797147,0.902265,0.562846,0.456642,0.199904,...,0.756926,0.928699,0.646722,0.961918,0.969270,0.850002,0.659250,0.148435,0.483421,0.900798
75%,7482.000000,0.907480,0.680696,0.023275,0.781415,0.852327,0.924124,0.597158,0.525858,0.310765,...,0.849786,0.940026,0.667928,0.968614,0.976805,0.870007,0.842969,0.193532,0.763943,0.914393
max,9640.000000,0.959442,0.831912,0.481823,0.961728,0.986409,0.962793,0.692725,0.740867,0.676890,...,0.928438,0.968532,0.759299,0.985654,0.988214,0.928761,0.954537,0.479137,0.973429,0.951438
